- Date Mon Oct 27 07:11PM to 07:28PM
- Verify the data with CAN_C2C data on ByteBeam dataset

In [2]:
import csv
from datetime import datetime

def csv_to_asc(input_csv, output_asc):
    # Read CSV and sort by timestamp then sequence
    with open(input_csv, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        rows = sorted(reader, key=lambda x: (int(x[1]), int(x[3])))
    
    # Generate ASC header from first timestamp
    first_ts = int(rows[0][1])
    dt = datetime.fromtimestamp(first_ts / 1000)
    header_date = dt.strftime("%a %b %d  %I:%M:%S %p %Y")
    
    # Process each row
    with open(output_asc, 'w') as f:
        # Write header
        f.write(f"date {header_date}\n")
        f.write("base hex timestamps absolute\n")
        
        for row in rows:
            # Calculate relative timestamp in seconds
            ts = (int(row[1]) - first_ts) / 1000.0
            
            # Convert CAN ID to 8-digit hex with 'x' suffix
            can_id = f"{int(row[8]):08x}x"
            
            # Rearrange bytes: byte1, byte2, byte3, byte4, byte5, byte6, byte7, byte8
            byte_order = [4, 10, 7, 9, 11, 6, 12, 5]
            data_bytes = ' '.join(f"{int(row[i]):02x}" for i in byte_order)
            
            # Write formatted line
            f.write(f"{ts:.6f} 1 {can_id} Rx d 8 {data_bytes}\n")

# Example usage
csv_to_asc('device_7_10thSept_Powertrain_Failure.csv', 'device_7_10thSept_Powertrain_Failure.asc')